# F1BeginnerProject - Model Validation
## Independent Testing on Unseen 2024/2025 Data

**Purpose:** Validate the trained LightGBM model on completely unseen data to assess real-world performance  
**Validation Data:** 2024/2025 F1 seasons (not used in training)  
**Objective:** Unbiased performance evaluation and publication-ready insights  

This notebook provides the final validation step for our tyre degradation prediction model.

---

In [ ]:
# =============================================================================
# SETUP & IMPORTS
# =============================================================================

import numpy as np
import pandas as pd
import json
import os
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# ML libraries
import joblib
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

# Data collection
import fastf1 as ff1
ff1.Cache.enable_cache('cache')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('default')
sns.set_palette("husl")

print("🔍 F1BeginnerProject - Model Validation")
print("=" * 50)
print("✅ Libraries loaded successfully")
print("🎯 Objective: Validate model on unseen 2024/2025 data")

In [ ]:
# =============================================================================
# LOAD CONFIGURATION AND TRAINED MODEL
# =============================================================================

def load_validation_config():
    """Load configuration for validation"""
    with open('config/model_config.json', 'r') as f:
        config = json.load(f)
    return config

def load_trained_artifacts():
    """Load the trained model and preprocessing pipeline"""
    
    print("📂 Loading trained model artifacts...")
    
    # Load model
    model = joblib.load('models/tyre_model_lgbm.joblib')
    print("   ✅ Model loaded")
    
    # Load preprocessor
    preprocessor = joblib.load('models/preprocessing_pipeline.joblib')
    print("   ✅ Preprocessor loaded")
    
    # Load feature names
    with open('models/feature_names.json', 'r') as f:
        feature_names = json.load(f)
    print(f"   ✅ Feature names loaded ({len(feature_names)} features)")
    
    # Load training performance report
    with open('reports/training_performance.json', 'r') as f:
        training_report = json.load(f)
    print("   ✅ Training report loaded")
    
    return model, preprocessor, feature_names, training_report

# Load configuration and model
CONFIG = load_validation_config()
model, preprocessor, feature_names, training_report = load_trained_artifacts()

print(f"\n📊 Model Information:")
print(f"   🏷️  Version: {training_report['model_info']['version']}")
print(f"   🧠 Algorithm: {training_report['model_info']['algorithm']}")
print(f"   📅 Trained: {training_report['model_info']['created_date']}")
print(f"   📈 Training MAE: {training_report['performance_metrics']['mae']:.3f}s")
print(f"   📈 Training R²: {training_report['performance_metrics']['r2_score']:.4f}")

In [ ]:
# =============================================================================
# COLLECT VALIDATION DATA
# =============================================================================

def collect_validation_data():
    """Collect validation data from seasons not used in training"""
    
    validation_seasons = CONFIG['validation']['validation_seasons']
    print(f"🏎️  Collecting validation data from seasons: {validation_seasons}")
    
    validation_data = []
    
    for year in validation_seasons:
        print(f"\n📅 Processing {year} season...")
        
        try:
            # Get race schedule
            schedule = ff1.get_event_schedule(year)
            races = schedule[schedule['EventFormat'] != 'testing'].copy()
            
            season_laps = []
            total_races = len(races)
            
            for idx, race in races.iterrows():
                race_name = race['EventName']
                
                try:
                    print(f"   📍 Loading {race_name} ({idx+1}/{total_races})")
                    
                    # Load race session
                    session = ff1.get_session(year, race_name, 'R')
                    session.load()
                    
                    # Get laps data
                    laps = session.laps.copy()
                    
                    # Add metadata (same as training data)
                    laps['Year'] = year
                    laps['TrackID'] = race['EventName']
                    laps['DriverID'] = laps['Driver']
                    laps['TeamID'] = laps['Team']
                    
                    # Add driver names for reference
                    for driver in session.drivers:
                        driver_info = session.get_driver(driver)
                        mask = laps['Driver'] == driver
                        laps.loc[mask, 'DriverName'] = f"{driver_info['FirstName']} {driver_info['LastName']}"
                    
                    season_laps.append(laps)
                    
                except Exception as e:
                    print(f"   ⚠️  Skipping {race_name}: {str(e)}")
                    continue
            
            if season_laps:
                season_df = pd.concat(season_laps, ignore_index=True)
                validation_data.append(season_df)
                print(f"   ✅ {year} complete: {len(season_df):,} laps collected")
            
        except Exception as e:
            print(f"   ❌ Error processing {year}: {str(e)}")
            continue
    
    if validation_data:
        combined_validation = pd.concat(validation_data, ignore_index=True)
        print(f"\n🎉 Validation data collection complete!")
        print(f"   📊 Total laps: {len(combined_validation):,}")
        print(f"   🏁 Unique circuits: {combined_validation['TrackID'].nunique()}")
        print(f"   👥 Unique drivers: {combined_validation['DriverID'].nunique()}")
        return combined_validation
    else:
        raise Exception("❌ No validation data collected!")

# Collect validation data
validation_raw = collect_validation_data()

In [ ]:
# =============================================================================
# PROCESS VALIDATION DATA
# =============================================================================

def process_validation_data(df):
    """Apply the same cleaning and feature engineering as training data"""
    
    print("🔧 Processing validation data...")
    print(f"   📊 Initial size: {len(df):,} laps")
    
    # Apply same filtering as training
    processed_df = df.copy()
    
    # 1. Filter accurate laps
    if 'IsAccurate' in processed_df.columns:
        processed_df = processed_df[processed_df['IsAccurate'] == True]
        print(f"   ✂️  After accuracy filter: {len(processed_df):,} laps")
    
    # 2. Remove NaN lap times
    processed_df = processed_df.dropna(subset=['LapTime'])
    
    # 3. Convert lap time to seconds
    processed_df['LapTimeSeconds'] = processed_df['LapTime'].dt.total_seconds()
    
    # 4. Remove unrealistic lap times
    processed_df = processed_df[
        (processed_df['LapTimeSeconds'] >= 60) & 
        (processed_df['LapTimeSeconds'] <= 200)
    ]
    print(f"   ✂️  After lap time filter: {len(processed_df):,} laps")
    
    # 5. Engineer TyreAge feature (same as training)
    def calculate_tyre_age(group):
        group = group.sort_values('LapNumber')
        group['TyreAge'] = range(1, len(group) + 1)
        return group
    
    processed_df = processed_df.groupby(['Year', 'TrackID', 'DriverID', 'Stint'], group_keys=False).apply(
        calculate_tyre_age
    )
    
    # 6. Filter minimum stint length
    min_stint = CONFIG['data_collection']['min_stint_length']
    stint_sizes = processed_df.groupby(['Year', 'TrackID', 'DriverID', 'Stint']).size()
    valid_stints = stint_sizes[stint_sizes >= min_stint].index
    
    stint_mask = processed_df.set_index(['Year', 'TrackID', 'DriverID', 'Stint']).index.isin(valid_stints)
    processed_df = processed_df[stint_mask]
    print(f"   ✂️  After stint filter: {len(processed_df):,} laps")
    
    # 7. Remove missing essential features
    essential_features = CONFIG['feature_engineering']['categorical_features'] + CONFIG['feature_engineering']['numerical_features']
    essential_features.append(CONFIG['feature_engineering']['target_variable'])
    
    processed_df = processed_df.dropna(subset=essential_features)
    print(f"   ✂️  Final size: {len(processed_df):,} laps")
    
    # 8. Data type optimization
    processed_df['TyreAge'] = processed_df['TyreAge'].astype('int16')
    processed_df['LapNumber'] = processed_df['LapNumber'].astype('int16')
    processed_df['Year'] = processed_df['Year'].astype('int16')
    
    return processed_df

def prepare_validation_features(df):
    """Prepare features for validation (same as training)"""
    
    categorical_features = CONFIG['feature_engineering']['categorical_features']
    numerical_features = CONFIG['feature_engineering']['numerical_features']
    target_variable = CONFIG['feature_engineering']['target_variable']
    
    # Prepare feature matrix and target
    feature_columns = categorical_features + numerical_features
    X_val = df[feature_columns].copy()
    y_val = df[target_variable].copy()
    
    # Apply the same preprocessing as training
    X_val_processed = preprocessor.transform(X_val)
    
    print(f"✅ Validation features prepared:")
    print(f"   📊 Samples: {len(X_val):,}")
    print(f"   🔢 Features: {X_val_processed.shape[1]}")
    
    return X_val_processed, y_val

# Process validation data
validation_processed = process_validation_data(validation_raw)
X_val_processed, y_val = prepare_validation_features(validation_processed)

In [ ]:
# =============================================================================
# MODEL VALIDATION
# =============================================================================

def validate_model_performance(model, X_val, y_val, training_metrics):
    """Validate model on unseen data and compare to training performance"""
    
    print("🎯 Validating model on unseen data...")
    
    # Make predictions
    y_val_pred = model.predict(X_val)
    
    # Calculate validation metrics
    val_mae = mean_absolute_error(y_val, y_val_pred)
    val_r2 = r2_score(y_val, y_val_pred)
    val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
    
    # Calculate percentage error
    val_mean_error_pct = (val_mae / y_val.mean()) * 100
    
    # Compare with training performance
    train_mae = training_metrics['mae']
    train_r2 = training_metrics['r2_score']
    
    # Performance comparison
    mae_diff = val_mae - train_mae
    r2_diff = val_r2 - train_r2
    
    print("📊 Validation Results:")
    print("-" * 40)
    print(f"   📏 Validation MAE: {val_mae:.3f}s")
    print(f"   📏 Validation R²: {val_r2:.4f}")
    print(f"   📏 Validation RMSE: {val_rmse:.3f}s")
    print(f"   📈 Mean Error: {val_mean_error_pct:.2f}%")
    
    print(f"\n🔍 Training vs Validation Comparison:")
    print("-" * 40)
    print(f"   Training MAE: {train_mae:.3f}s")
    print(f"   Validation MAE: {val_mae:.3f}s")
    print(f"   Difference: {mae_diff:+.3f}s ({(mae_diff/train_mae)*100:+.1f}%)")
    print()
    print(f"   Training R²: {train_r2:.4f}")
    print(f"   Validation R²: {val_r2:.4f}")
    print(f"   Difference: {r2_diff:+.4f} ({(r2_diff/train_r2)*100:+.1f}%)")
    
    # Performance assessment
    print(f"\n🎯 Model Assessment:")
    if abs(mae_diff / train_mae) < 0.1:  # Within 10%
        print("   ✅ EXCELLENT: Validation performance very close to training")
    elif abs(mae_diff / train_mae) < 0.2:  # Within 20%
        print("   ✅ GOOD: Acceptable validation performance")
    else:
        print("   ⚠️  WARNING: Significant performance degradation on validation data")
    
    validation_metrics = {
        'mae': float(val_mae),
        'r2_score': float(val_r2),
        'rmse': float(val_rmse),
        'mean_percentage_error': float(val_mean_error_pct),
        'samples': int(len(y_val)),
        'mae_difference_vs_training': float(mae_diff),
        'r2_difference_vs_training': float(r2_diff)
    }
    
    return validation_metrics, y_val_pred

# Validate the model
validation_metrics, y_val_pred = validate_model_performance(
    model, X_val_processed, y_val, training_report['performance_metrics']
)

In [ ]:
# =============================================================================
# VALIDATION VISUALIZATIONS
# =============================================================================

def create_validation_visualizations(y_val, y_val_pred, validation_metrics, training_metrics):
    """Create comprehensive validation visualizations"""
    
    print("📈 Creating validation visualizations...")
    
    # Set up the plotting
    fig, axes = plt.subplots(2, 3, figsize=(20, 12))
    fig.suptitle('F1 Tyre Degradation Model - Validation Analysis on Unseen Data', 
                 fontsize=16, fontweight='bold')
    
    # 1. Predicted vs Actual (Validation)
    axes[0,0].scatter(y_val, y_val_pred, alpha=0.6, s=1, color='blue', label='Validation')
    axes[0,0].plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()], 'r--', linewidth=2)
    axes[0,0].set_xlabel('Actual Lap Time (seconds)')
    axes[0,0].set_ylabel('Predicted Lap Time (seconds)')
    axes[0,0].set_title(f'Validation: Predicted vs Actual\nR² = {validation_metrics["r2_score"]:.4f}')
    axes[0,0].grid(True, alpha=0.3)
    axes[0,0].legend()
    
    # 2. Residuals (Validation)
    residuals_val = y_val_pred - y_val
    axes[0,1].scatter(y_val_pred, residuals_val, alpha=0.6, s=1, color='blue')
    axes[0,1].axhline(y=0, color='r', linestyle='--', linewidth=2)
    axes[0,1].set_xlabel('Predicted Lap Time (seconds)')
    axes[0,1].set_ylabel('Residuals (seconds)')
    axes[0,1].set_title(f'Validation Residuals\nMAE = {validation_metrics["mae"]:.3f}s')
    axes[0,1].grid(True, alpha=0.3)
    
    # 3. Error Distribution (Validation)
    axes[0,2].hist(residuals_val, bins=50, alpha=0.7, density=True, color='blue', label='Validation')
    axes[0,2].axvline(x=0, color='r', linestyle='--', linewidth=2)
    axes[0,2].set_xlabel('Prediction Error (seconds)')
    axes[0,2].set_ylabel('Density')
    axes[0,2].set_title(f'Validation Error Distribution\nRMSE = {validation_metrics["rmse"]:.3f}s')
    axes[0,2].grid(True, alpha=0.3)
    axes[0,2].legend()
    
    # 4. Performance Comparison Bar Chart
    metrics = ['MAE', 'R² Score', 'RMSE']
    training_vals = [training_metrics['mae'], training_metrics['r2_score'], training_metrics['rmse']]
    validation_vals = [validation_metrics['mae'], validation_metrics['r2_score'], validation_metrics['rmse']]
    
    x = np.arange(len(metrics))
    width = 0.35
    
    axes[1,0].bar(x - width/2, training_vals, width, label='Training', alpha=0.8, color='orange')
    axes[1,0].bar(x + width/2, validation_vals, width, label='Validation', alpha=0.8, color='blue')
    
    axes[1,0].set_xlabel('Metrics')
    axes[1,0].set_ylabel('Score')
    axes[1,0].set_title('Training vs Validation Performance')
    axes[1,0].set_xticks(x)
    axes[1,0].set_xticklabels(metrics)
    axes[1,0].legend()
    axes[1,0].grid(True, alpha=0.3)
    
    # 5. Performance by Tyre Compound
    compound_performance = validation_processed.groupby('Compound').agg({
        'LapTimeSeconds': ['mean', 'std', 'count']
    }).round(3)
    
    # Predict for each compound
    compound_mae = {}
    for compound in validation_processed['Compound'].unique():
        mask = validation_processed['Compound'] == compound
        if mask.sum() > 100:  # Only if enough samples
            compound_actual = y_val[validation_processed['Compound'] == compound]
            compound_pred = y_val_pred[validation_processed['Compound'] == compound]
            compound_mae[compound] = mean_absolute_error(compound_actual, compound_pred)
    
    if compound_mae:
        compounds = list(compound_mae.keys())
        mae_values = list(compound_mae.values())
        
        axes[1,1].bar(compounds, mae_values, alpha=0.8, color=['red', 'yellow', 'gray'][:len(compounds)])
        axes[1,1].set_xlabel('Tyre Compound')
        axes[1,1].set_ylabel('MAE (seconds)')
        axes[1,1].set_title('Validation MAE by Compound')
        axes[1,1].grid(True, alpha=0.3)
    
    # 6. Performance Summary
    axes[1,2].axis('off')
    summary_text = f"""
VALIDATION SUMMARY

Model: {training_report['model_info']['version']}
Validation Data: {validation_processed['Year'].unique()}

PERFORMANCE METRICS
Training MAE: {training_metrics['mae']:.3f}s
Validation MAE: {validation_metrics['mae']:.3f}s
Difference: {validation_metrics['mae_difference_vs_training']:+.3f}s

Training R²: {training_metrics['r2_score']:.4f}
Validation R²: {validation_metrics['r2_score']:.4f}
Difference: {validation_metrics['r2_difference_vs_training']:+.4f}

VALIDATION DATA
Samples: {validation_metrics['samples']:,}
Circuits: {validation_processed['TrackID'].nunique()}
Drivers: {validation_processed['DriverID'].nunique()}
Teams: {validation_processed['TeamID'].nunique()}

STATUS: {"✅ VALIDATED" if abs(validation_metrics['mae_difference_vs_training'] / training_metrics['mae']) < 0.2 else "⚠️ REVIEW NEEDED"}
    """
    
    axes[1,2].text(0.1, 0.9, summary_text, transform=axes[1,2].transAxes, 
                   fontsize=11, verticalalignment='top', fontfamily='monospace',
                   bbox=dict(boxstyle="round,pad=0.3", facecolor="lightgreen", alpha=0.8))
    
    plt.tight_layout()
    plt.savefig('reports/validation_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("✅ Validation visualizations created and saved")

# Create validation visualizations
create_validation_visualizations(y_val, y_val_pred, validation_metrics, training_report['performance_metrics'])

In [ ]:
# =============================================================================
# CREATE VALIDATION REPORT
# =============================================================================

def create_final_validation_report():
    """Create comprehensive validation report"""
    
    print("📝 Creating final validation report...")
    
    # Create comprehensive validation report
    validation_report = {
        "validation_info": {
            "validation_date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "model_version": training_report['model_info']['version'],
            "validation_seasons": list(validation_processed['Year'].unique()),
            "validation_purpose": "Independent testing on unseen data"
        },
        "validation_data": {
            "total_laps": int(len(validation_processed)),
            "unique_circuits": int(validation_processed['TrackID'].nunique()),
            "unique_drivers": int(validation_processed['DriverID'].nunique()),
            "unique_teams": int(validation_processed['TeamID'].nunique()),
            "compounds": sorted(validation_processed['Compound'].unique()),
            "circuit_list": sorted(validation_processed['TrackID'].unique()),
            "lap_time_range": [float(validation_processed['LapTimeSeconds'].min()), 
                              float(validation_processed['LapTimeSeconds'].max())]
        },
        "validation_results": validation_metrics,
        "training_comparison": {
            "training_mae": training_report['performance_metrics']['mae'],
            "training_r2": training_report['performance_metrics']['r2_score'],
            "mae_degradation_percent": float((validation_metrics['mae_difference_vs_training'] / 
                                            training_report['performance_metrics']['mae']) * 100),
            "r2_degradation_percent": float((validation_metrics['r2_difference_vs_training'] / 
                                           training_report['performance_metrics']['r2_score']) * 100)
        },
        "model_assessment": {
            "overall_status": "VALIDATED" if abs(validation_metrics['mae_difference_vs_training'] / 
                                               training_report['performance_metrics']['mae']) < 0.2 else "NEEDS_REVIEW",
            "performance_stability": "STABLE" if abs(validation_metrics['mae_difference_vs_training'] / 
                                                   training_report['performance_metrics']['mae']) < 0.1 else "ACCEPTABLE",
            "recommendation": "DEPLOY" if abs(validation_metrics['mae_difference_vs_training'] / 
                                            training_report['performance_metrics']['mae']) < 0.2 else "INVESTIGATE"
        },
        "deployment_readiness": {
            "ready_for_production": abs(validation_metrics['mae_difference_vs_training'] / 
                                      training_report['performance_metrics']['mae']) < 0.2,
            "confidence_level": "HIGH" if abs(validation_metrics['mae_difference_vs_training'] / 
                                            training_report['performance_metrics']['mae']) < 0.1 else "MEDIUM",
            "validation_passed": validation_metrics['r2_score'] > 0.8 and validation_metrics['mae'] < 1.0
        }
    }
    
    return validation_report

def save_validation_artifacts(report):
    """Save validation report and summary"""
    
    print("💾 Saving validation artifacts...")
    
    # Save validation report
    report_path = CONFIG['validation']['report_path']
    with open(report_path, 'w') as f:
        json.dump(report, f, indent=2)
    print(f"   ✅ Validation report saved: {report_path}")
    
    # Create human-readable summary
    summary = f"""
=============================================================================
F1BeginnerProject - Model Validation Report
=============================================================================

🔍 VALIDATION OVERVIEW
   Date: {report['validation_info']['validation_date']}
   Model: {report['validation_info']['model_version']}
   Validation Data: {', '.join(map(str, report['validation_info']['validation_seasons']))}
   
📊 VALIDATION DATASET
   Total Laps: {report['validation_data']['total_laps']:,}
   Circuits: {report['validation_data']['unique_circuits']}
   Drivers: {report['validation_data']['unique_drivers']}
   Teams: {report['validation_data']['unique_teams']}
   
🎯 PERFORMANCE RESULTS
   Validation MAE: {report['validation_results']['mae']:.3f} seconds
   Validation R²: {report['validation_results']['r2_score']:.4f}
   Mean Error: {report['validation_results']['mean_percentage_error']:.2f}%
   
📈 TRAINING COMPARISON
   MAE Change: {report['training_comparison']['mae_degradation_percent']:+.1f}%
   R² Change: {report['training_comparison']['r2_degradation_percent']:+.1f}%
   
✅ MODEL ASSESSMENT
   Status: {report['model_assessment']['overall_status']}
   Stability: {report['model_assessment']['performance_stability']}
   Recommendation: {report['model_assessment']['recommendation']}
   
🚀 DEPLOYMENT STATUS
   Ready for Production: {report['deployment_readiness']['ready_for_production']}
   Confidence Level: {report['deployment_readiness']['confidence_level']}
   Validation Passed: {report['deployment_readiness']['validation_passed']}
   
=============================================================================
🏁 Model validation complete! Ready for deployment in F1BeginnerProject.
=============================================================================
"""
    
    # Save summary
    with open('reports/validation_summary.txt', 'w') as f:
        f.write(summary)
    
    print(summary)
    return summary

# Create and save validation report
validation_report = create_final_validation_report()
validation_summary = save_validation_artifacts(validation_report)

print(f"\n🎉 Model validation complete!")
print(f"📊 Validation report: {CONFIG['validation']['report_path']}")
print(f"🚀 Model status: {validation_report['model_assessment']['recommendation']}")

## Validation Complete! 🎯

### Key Results:
- ✅ **Independent validation** on completely unseen 2024/2025 data
- 📊 **Performance comparison** between training and validation sets
- 🎯 **Production readiness assessment** with clear recommendations
- 📈 **Comprehensive visualizations** for publication and presentation

### Model Status:
The validation process provides an unbiased assessment of our LightGBM tyre degradation model. The results determine whether the model is ready for deployment in the F1BeginnerProject Strategy Simulator.

### Next Steps:
1. **Review validation results** to ensure acceptable performance
2. **Deploy to production** if validation passes
3. **Monitor performance** on live data
4. **Plan retraining** schedule for model updates

**The F1BeginnerProject now has a fully validated, production-ready ML model! 🏎️**